# Azure AI Search as Agent Memory Demo (Vinod traveling to Seattle)

This notebook shows how **Mem0** can use **Azure AI Search** for storing agent memory, combined with **Azure OpenAI** to generate answers. Our user **Vinod** wants to travel from **San Francisco** to **Seattle**, is vegetarian, and has a peanut allergy.

We’ll demonstrate:
1. **User/Semantic Memory** (long-term, semantic) with access control
2. **Session/Episodic Memory** (short-term, run-based)
3. **Agent/Procedural Memory** (procedural instructions)
4. **Multi-Agent** usage (travel & dining assistants)
5. **Summarization** of a large transcript
6. **Retrieving Agent Memory for Agentic Grounding** for final output

## 1) Installation

```bash
pip install openai mem0ai azure-search-documents==11.5.2 python-dotenv

In [ ]:
!pip install openai 
!pip install azure-search-documents==11.5.2
!pip install mem0ai --pre
!pip install python-dotenv

## 2) Imports & Environment Setup

In [ ]:
import os
import openai
from dotenv import load_dotenv
from mem0 import Memory, MemoryClient
from openai import AzureOpenAI

# Load environment from .env or system
load_dotenv()

# Azure OpenAI configuration
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
AZURE_OPENAI_CHAT_COMPLETION_DEPLOYED_MODEL_NAME = os.getenv("AZURE_OPENAI_CHAT_COMPLETION_DEPLOYED_MODEL_NAME")
AZURE_OPENAI_EMBEDDING_DEPLOYED_MODEL_NAME = os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYED_MODEL_NAME")

# Azure AI Search
SEARCH_SERVICE_ENDPOINT = os.getenv("AZURE_SEARCH_SERVICE_ENDPOINT")
SEARCH_SERVICE_API_KEY = os.getenv("AZURE_SEARCH_ADMIN_KEY")

# Mem0
MEM0AI_API_KEY = os.getenv("MEM0AI_API_KEY")

# Configure OpenAI for Azure
client = AzureOpenAI(
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    api_key=AZURE_OPENAI_API_KEY,
    api_version="2024-12-01-preview",
)

print("Environment loaded.")


Environment loaded.


## 3) Configure Mem0 with Azure AI Search

In [36]:
# Initialize Mem0 with Azure OpenAI and Azure AI Search integration
config = {
    "llm": {
        "provider": "azure_openai",
        "config": {
            "model": AZURE_OPENAI_CHAT_COMPLETION_DEPLOYED_MODEL_NAME,
            "temperature": 0.1,
            "max_tokens": 2000,
            "azure_kwargs": {
                "azure_deployment": AZURE_OPENAI_CHAT_COMPLETION_DEPLOYED_MODEL_NAME,
                "api_version": "2024-10-21",
                "azure_endpoint": AZURE_OPENAI_ENDPOINT,
                "api_key": AZURE_OPENAI_API_KEY,
            },
        },
    },
    "embedder": {
        "provider": "azure_openai",
        "config": {
            "model": AZURE_OPENAI_EMBEDDING_DEPLOYED_MODEL_NAME,
            "embedding_dims": 1536,
            "azure_kwargs": {
                "api_version": "2024-10-21",
                "azure_deployment": AZURE_OPENAI_EMBEDDING_DEPLOYED_MODEL_NAME,
                "azure_endpoint": AZURE_OPENAI_ENDPOINT,
                "api_key": AZURE_OPENAI_API_KEY,
            },
        },
    },
    "vector_store": {
        "provider": "azure_ai_search",
        "config": {
            "service_name": "fsunavala-performance-test",
            "api_key": SEARCH_SERVICE_API_KEY,
            "collection_name": "aisearch-agent-memory-poc",
            "embedding_model_dims": 1536,
            "use_compression": True,
        },
    },
    "version": "v1.1",  
}

memory = Memory.from_config(config)
mem0_client = MemoryClient(api_key=MEM0AI_API_KEY)

## 4) Simple GPT-4o Chat Helper

In [46]:
import openai

# We'll maintain a conversation log to simulate a small chat
conversation_log = [
    {"role": "system", "content": "You are a friendly travel planning assistant for Seattle trips."},
]

def ask_gpt(user_message, temperature=0.7):
    conversation_log.append({"role": "user", "content": user_message})
    response = client.chat.completions.create(
        model=AZURE_OPENAI_CHAT_COMPLETION_DEPLOYED_MODEL_NAME,
        messages=conversation_log,
        temperature=temperature
    )
    reply = response.choices[0].message.content
    conversation_log.append({"role": "assistant", "content": reply})
    return reply

print("ask_gpt function created.")


ask_gpt function created.


## 5) Scenario A: User Memory + Access Control
Vinod is our user. He’s traveling from SF to Seattle, is vegetarian, and has a peanut allergy.

In [38]:
user_id = "vinod"

# Public preference
memory.add(
    "I am allergic to peanuts and prefer vegetarian meals.",
    user_id=user_id,
    metadata={"access_level": "public"}
)

# Private note
memory.add(
    "I have an upcoming medical procedure on March 1st, prefer to keep it private.",
    user_id=user_id,
    metadata={"access_level": "private"}
)

print("Scenario A user memory (Vinod) stored. Searching...")

query = "What do we know about Vinod's preferences?"
results_user = memory.search(query, user_id=user_id)
for r in results_user.get("results", []):
    print(f"- ID: {r['id']} | Memory: {r['memory']} | Score: {r.get('score')}")


Scenario A user memory (Vinod) stored. Searching...
- ID: 03abb7cb-c2c3-4cd1-8d07-8754422b418c | Memory: Prefers vegetarian meals | Score: 0.61694175
- ID: 86db5147-9178-4984-8424-c055fc108358 | Memory: Prefers to keep the procedure private | Score: 0.58864707
- ID: 5721d8d5-5ae6-4ad1-b1e9-7d6fdc4854bb | Memory: Allergic to peanuts | Score: 0.54264903
- ID: 66e75c20-a332-461e-9004-683258cebc81 | Memory: Upcoming medical procedure on March 1st | Score: 0.5184054


## 6) Scenario B: Session Memory (Episodic)
Vinod is planning a Seattle trip (flying from SF). We store ephemeral details under run_id=vinod-seattle-2024.

In [39]:
session_id = "vinod-seattle-2024"

memory.add(
    "Vinod wants a 3-star or better hotel in downtown Seattle. Budget is moderate.",
    user_id=user_id,
    run_id=session_id
)

memory.add(
    "Vinod also wants to explore the Space Needle, Pike Place Market, and vegetarian restaurants near them.",
    user_id=user_id,
    run_id=session_id
)

print("Session memory stored for Seattle trip.")

search_query = "What do we know about the hotel rating preference?"
session_results = memory.search(search_query, user_id=user_id, run_id=session_id)
print("Session Memory Search:")
for r in session_results.get("results", []):
    print(f"- ID: {r['id']} | Memory: {r['memory']}")


Session memory stored for Seattle trip.
Session Memory Search:
- ID: a46c4eb8-b7ca-45f6-8448-4fc1d8dfb60f | Memory: Wants a 3-star or better hotel in downtown Seattle
- ID: 03abb7cb-c2c3-4cd1-8d07-8754422b418c | Memory: Prefers vegetarian meals
- ID: 86db5147-9178-4984-8424-c055fc108358 | Memory: Prefers to keep the procedure private
- ID: e3b1ce5e-d365-43c4-bb55-15e5181f5563 | Memory: Budget is moderate
- ID: f024b4e5-53f8-499c-89cc-8c738d40812b | Memory: Wants to explore the Space Needle
- ID: 5721d8d5-5ae6-4ad1-b1e9-7d6fdc4854bb | Memory: Allergic to peanuts
- ID: 66e75c20-a332-461e-9004-683258cebc81 | Memory: Upcoming medical procedure on March 1st


## 7) Scenario C: Agent (Procedural) Memory + Multi-Agent
We have two agents:

- **travel-assistant**
- **dining-assistant**

We store their procedural instructions. Then we store a multi-agent memory indicating peanut-free constraints.

In [41]:
agent_travel = "travel-assistant"
agent_dining = "dining-assistant"

# Travel agent rules
memory.add(
    "Always greet Vinod politely and consider flights from SF to Seattle + hotel rating.",
    agent_id=agent_travel,
    metadata={"priority": "high"}
)

# Dining agent rules
memory.add(
    "Offer at least 3 vegetarian recommendations if user wants restaurants. Avoid peanuts.",
    agent_id=agent_dining,
    metadata={"priority": "high"}
)

print("Agent memory stored for both travel/dining assistants.")

# Multi-agent memory
multi_text = "Both travel and dining must exclude peanuts for Vinod, who is severely allergic."
memory.add(
    multi_text,
    agent_id=[agent_travel, agent_dining],
    metadata={"category": "safety"}
)
print("Multi-agent memory added for travel & dining.")


Agent memory stored for both travel/dining assistants.
Multi-agent memory added for travel & dining.


### Check multi-agent memory

In [44]:
peanut_query = "peanuts"
dining_search = memory.search(peanut_query, agent_id=agent_dining, limit=3) # adding limit=3 (fyi - same as top=3 in AZS request body) here for demo but not required
print("Dining sees:\n", dining_search)

travel_search = memory.search(peanut_query, agent_id=agent_travel, limit=3)
print("\nTravel sees:\n", travel_search)


Dining sees:
 {'results': [{'id': '8766bfb8-7cc6-4230-8da4-fac4be331d9c', 'memory': 'User wants to avoid peanuts', 'hash': '316eb3df8ec9cd1aa31a00f1c0ef1a8a', 'metadata': {'priority': 'high'}, 'score': 0.6816331, 'created_at': '2025-01-25T16:02:23.501853-08:00', 'updated_at': None, 'agent_id': 'dining-assistant'}, {'id': '5721d8d5-5ae6-4ad1-b1e9-7d6fdc4854bb', 'memory': 'Travel and dining must exclude peanuts for Vinod, who is severely allergic', 'hash': 'f82247f101377141406c71223aec2897', 'metadata': {'category': 'safety'}, 'score': 0.5984967, 'created_at': '2025-01-25T15:59:38.554638-08:00', 'updated_at': '2025-01-25T16:02:00.605947-08:00', 'user_id': 'vinod', 'agent_id': ['travel-assistant', 'dining-assistant']}, {'id': '49b7bfa7-a2ae-4f2e-a9bc-ceefa40cddf0', 'memory': 'Wants to explore Pike Place Market', 'hash': 'b4c37a1834d69dc35c908ae93cf080c6', 'metadata': None, 'score': 0.5632208, 'created_at': '2025-01-25T16:01:13.988585-08:00', 'updated_at': None, 'user_id': 'vinod', 'run_id

## 8) Summarization Example
We’ll define a large conversation about traveling to Seattle, ask GPT to summarize, then store that summary in Mem0.

In [47]:
large_convo = """
User: Vinod, an avid traveler with a passion for exploring new cities, is gearing up for his journey from San Francisco to Seattle. As a dedicated vegetarian who also contends with a peanut allergy, Vinod carefully plans his meals to ensure both safety and satisfaction throughout his trip. Understanding the importance of comfortable accommodations, he is specifically looking for a 3-star or higher-rated hotel situated in the bustling downtown area of Seattle, where he can easily access his points of interest and immerse himself in the city's vibrant atmosphere.
Assistant: Thanks for providing such detailed information about Vinod’s preferences! To ensure his trip is well-planned, I’ll help him find a suitable hotel in downtown Seattle that meets his criteria. Given that Vinod is looking for vegetarian-friendly dining options and has a peanut allergy, I'll make sure to focus on areas with restaurants that cater to these needs. I’ll also look into ensuring the hotel offers comfortable amenities for a smooth stay. I'll check for options that are near his must-visit landmarks: the Space Needle and Pike Place Market.
User: During his stay, Vinod is excited to visit some of Seattle's most renowned attractions, including the iconic Space Needle and the lively Pike Place Market. These landmarks not only offer stunning views and unique experiences but also provide ample opportunities for Vinod to engage with the local culture and community. In addition to sightseeing, he is keen on discovering a variety of vegetarian-friendly restaurants that cater to his dietary needs without compromising on flavor or quality. Vinod values places that prioritize fresh, wholesome ingredients and offer creative vegetarian dishes, ensuring that his dining experiences are both enjoyable and safe.
Assistant: Excellent! The Space Needle and Pike Place Market are definitely must-see spots for Vinod, and they’re both located in prime areas of downtown Seattle, so we’ll focus on hotels nearby for easy access. As for vegetarian dining, Seattle is well-known for its diverse and inclusive food scene, so there will be no shortage of places catering to his dietary needs. Some great options include Plum Bistro, Araya's Place, and Cafe Flora—all known for their fresh, creative vegetarian menus. I’ll cross-reference these with the hotels we’re considering to ensure both convenience and culinary satisfaction for Vinod.
User: Beyond his culinary and sightseeing plans, Vinod has a deep appreciation for scenic views and nature. He intends to spend time exploring Seattle's picturesque parks, waterfront areas, and perhaps even take a leisurely stroll along the waterfront to soak in the natural beauty of the region. Whether it's watching the sunset over Elliott Bay or enjoying the serene ambiance of Kerry Park, Vinod seeks out moments that allow him to relax and connect with the environment around him.
Assistant: What a beautiful vision for his trip! Seattle offers some incredible scenic spots that align perfectly with Vinod’s love for nature. Kerry Park, in particular, is famous for its panoramic views of the Seattle skyline, the Space Needle, and Mount Rainier in the background—perfect for a relaxing moment. Discovery Park, with its trails and coastal views, is also a great option for Vinod to connect with nature. I’d suggest including some time for a waterfront stroll along Elliott Bay as well, with a visit to Olympic Sculpture Park, which combines art and nature in a very tranquil setting. These will provide great opportunities for scenic relaxation throughout his stay.
User: Balancing his desire for memorable experiences with budget considerations, Vinod is strategic in his spending, aiming to make the most of his trip without overshooting his financial plans. He meticulously researches affordable yet high-quality dining options, seeks out deals on attractions, and prioritizes activities that offer the best value for his investment. This thoughtful approach ensures that Vinod can enjoy a fulfilling and enriching visit to Seattle, experiencing the city's highlights while maintaining a comfortable and manageable budget.
Assistant: It’s great that Vinod is keeping a balanced approach when it comes to budgeting for his trip! Seattle offers many activities and dining options that won’t break the bank. For example, Pike Place Market itself is free to explore, and it offers plenty of food stalls where he can find affordable vegetarian-friendly options. The Seattle Center is home to the Space Needle and Museum of Pop Culture (MoPOP), but the nearby Chihuly Garden and Glass and the Seattle Center grounds can be visited for free. I’ll make sure to recommend a range of dining options that fit within a moderate budget—many of Seattle's best vegetarian-friendly restaurants offer great value for their quality. We’ll also find some great deals on tickets for the Space Needle, and if he’s looking for additional activities, Seattle Art Museum and Ferry rides to Bainbridge Island are excellent low-cost options.
User: In summary, Vinod's trip to Seattle is thoughtfully curated to align with his vegetarian lifestyle, allergy requirements, and love for scenic and cultural experiences. By selecting the right accommodations, planning visits to key landmarks, exploring diverse dining options, and managing his budget wisely, Vinod is set to have a rewarding and enjoyable adventure in one of the Pacific Northwest's most dynamic cities.
Assistant: Absolutely, Vinod’s trip is shaping up to be a wonderfully balanced experience. With his dietary restrictions and allergy needs taken into consideration, combined with his love for nature and scenic views, Seattle will provide plenty of opportunities for both relaxation and adventure. I’ll finalize the hotel recommendations in the downtown area, ensuring they’re close to his points of interest and offer safe dining options. Once we have the accommodations sorted, we’ll also map out a tailored itinerary with a mix of iconic attractions, nature spots, and great restaurants. This approach will ensure a fulfilling and budget-friendly adventure in Seattle.
"""

summary_prompt = f"Summarize this conversation in 2 sentences:\n\n{large_convo}"
summary_result = ask_gpt(summary_prompt)

memory.add(
    messages=summary_result,
    user_id=user_id,
    run_id="vinod-long-seattle",
    metadata={"type": "conversation_summary"}
)

print("Summary stored in Mem0:\n", summary_result)


Summary stored in Mem0:
 Vinod is planning a well-rounded trip to Seattle, focusing on vegetarian dining options that accommodate his peanut allergy, while also ensuring comfortable accommodations in downtown near key attractions like the Space Needle and Pike Place Market. He aims to balance memorable experiences with budget considerations, exploring scenic parks and waterfront areas to connect with nature and enjoy the local culture.


## 9) Combine Mem0 Search + GPT for Final Prompt

In [48]:
search_query = "peanut AND vegetarian"
search_results = memory.search(search_query, user_id=user_id)

bullets = []
for r in search_results.get("results", []):
    bullets.append("- " + r["memory"])

ctx = "\n".join(bullets) if bullets else "(No memory found)."

final_prompt = f"""
Vinod wants dinner recommendations in Seattle. 
Mem0 reveals:
{ctx}

Suggest a vegetarian, peanut-free restaurant near a 3-star hotel downtown.
"""

final_answer = ask_gpt(final_prompt)
print("\nFinal Agent Answer:\n", final_answer)



Final Agent Answer:
 Hello Vinod! I’m excited to help you find some great dining options during your trip to Seattle. One excellent vegetarian restaurant that is also peanut-free and located near the downtown area is **Plum Bistro**. 

Plum Bistro offers a creative menu featuring delicious vegetarian dishes made with fresh ingredients. It’s located not too far from both the Space Needle and Pike Place Market, making it a convenient choice for your dining experience.

For your stay, I recommend looking into **The Paramount Hotel**. It is a 3-star hotel situated in downtown Seattle, close to major attractions, and would provide comfortable accommodations during your visit. 

Let me know if you need more recommendations or assistance with your trip planning!
